In [1]:
import numpy as np
import cv2 as cv
import glob
 
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((7*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:7].T.reshape(-1,2)
objp=0.008*objp

In [2]:
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
 
images = glob.glob('*.jpg')
 
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (7,7), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (5,5), (-1,-1), criteria)
        imgpoints.append(corners2)
    # Draw and display the corners
    cv.drawChessboardCorners(img, (7,7), corners2, ret)
    cv.imshow('img', img)
    cv.waitKey(500)

cv.destroyAllWindows()

In [3]:
images

['19f090b2abf3c42f8bc4f361bbf4b323.jpg',
 'c4bf2e530c0c0e4e9a5af7fa1a81d356.jpg',
 '224716ff3deeadfe6a578730d1c31a34.jpg',
 'e7ccbaaafc5da0324a39da29bf5c9d72.jpg',
 '12d8a0bf774dd67e4a0724e8bacffae5.jpg',
 '7839c5bc7cb9da17edb35af1a1943fc4.jpg',
 '612ed1b9fcd509196547952e11bcddb5.jpg',
 'ba93d2601b3e5de8ad8b5ccf4bcf9e2e.jpg',
 '9eb21244062871276e556949cb4e7146.jpg',
 '683af129051c7db5b1131049022195ef.jpg',
 '071e795eabe63f778dd9ff7530d7b124.jpg',
 'd79fd69b6d2d57b6f69e02146958829d.jpg',
 '53ed0daa055b835d7dba6e3ea5812242.jpg',
 '14213327bbb8e077bf130990bd4771dd.jpg']

In [3]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [4]:
mtx

array([[919.21981864,   0.        , 356.41270451],
       [  0.        , 913.16565134, 236.9305    ],
       [  0.        ,   0.        ,   1.        ]])

In [5]:
dist

array([[-4.32733568e-01,  1.08659933e+00,  1.11155166e-03,
        -3.70676225e-04, -3.75586346e+00]])

In [6]:
#去畸变

In [7]:
img = cv.imread(images[0])
h, w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [8]:
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)

True

In [9]:
images[0]

'19f090b2abf3c42f8bc4f361bbf4b323.jpg'

In [10]:
#rvecs, tvecs的作用

In [11]:
def draw(img, corners, imgpts):
 corner = tuple(corners[0].ravel().astype(int))
 img = cv.line(img, corner , tuple(imgpts[0].ravel().astype(int)), (255,0,0), 5)
 img = cv.line(img, corner, tuple(imgpts[1].ravel().astype(int)), (0,255,0), 5)
 img = cv.line(img, corner , tuple(imgpts[2].ravel().astype(int)), (0,0,255), 5)
 return img

In [12]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((7*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:7].T.reshape(-1,2)
 
axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)

In [ ]:
img = cv.imread(images[0])
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
ret, corners = cv.findChessboardCorners(gray, (7,7),None)
 
if ret == True:
    corners2 = cv.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)

# Find the rotation and translation vectors.
ret,rvecs, tvecs = cv.solvePnP(objp, corners2, mtx, dist)

# project 3D points to image plane
imgpts, jac = cv.projectPoints(axis, rvecs, tvecs, mtx, dist)

img = draw(img,corners2 ,imgpts)
cv.imshow('img',img)
k = cv.waitKey(0) & 0xFF
if k == ord('s'):
    cv.imwrite(fname[:6]+'.png', img)
 
cv.destroyAllWindows()

In [15]:
rvecs, tvecs

(array([[-0.24406926],
        [ 0.00712551],
        [ 0.16918039]]),
 array([[-3.82911151],
        [-3.74079138],
        [22.01701187]]))